<a href="https://colab.research.google.com/github/kahtye/NLP--IMDB-Sentiment-Analysis/blob/main/NLP_Assignment2.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>


###Question 2 - step 3 and 4
How does this model compare to the above models?

This model as shown from the loss values is showing better results than the previous results in the one above. This is due to the factors such as hidden layers and using other functions to help the training functionality of it. As we can see, with the loss rate from the beginning is already significantly better than the previous. As it has better functions to help train the model itself and with the additionn of the hidden layers, we are able to train it better as it is being passed through the layers. 

#### Question 3:
How do you pick the "best" model -- what metric do you want to use here?

To pick the best model, we first researched different accuracy metrics. There are two that were considered: Mean Squared Error and R^2. The best model is the one with the lowest Mean Squared Error and the highest R^2. For our model selection purposes, we simply used Mean Squared Error as our indicator. Upon testing with different parameters, it was concluded that a Logistic Model was the best using Unigrams with a MSE of 0.25.



In this assignment, you will familiarize yourself with:

    spaCy
    numpy
    PyTorch

to develop

    Multiple sentiment analysis models

to be able to

    Predict whether a given review is positive or negative

Before we begin, make sure your runtime is set to GPU -- 

Runtime > Change runtime type set to GPU

First we will load in some data.

Provided is code that will download a file and rename it to reviews.tar.gz


In [ ]:

!wget http://ai.stanford.edu/~amaas/data/sentiment/aclImdb_v1.tar.gz 


--2021-03-05 03:42:27--  http://ai.stanford.edu/~amaas/data/sentiment/aclImdb_v1.tar.gz
Resolving ai.stanford.edu (ai.stanford.edu)... 171.64.68.10
Connecting to ai.stanford.edu (ai.stanford.edu)|171.64.68.10|:80... connected.
HTTP request sent, awaiting response... 200 OK
Length: 84125825 (80M) [application/x-gzip]
Saving to: ‘aclImdb_v1.tar.gz.2’

aclImdb_v1.tar.gz.2 100%[===================>]  80.23M  25.7MB/s    in 3.3s    

2021-03-05 03:42:31 (24.0 MB/s) - ‘aclImdb_v1.tar.gz.2’ saved [84125825/84125825]



In [ ]:
!tar -xf aclImdb_v1.tar.gz

Included is a vocabulary of all the words we can potentially see -- No '\<UNK\>' *here*

In [ ]:
!head aclImdb/imdb.vocab

the
and
a
of
to
is
it
in
i
this


In [ ]:
positive_reviews = !ls -1 -d aclImdb/train/pos/*
negative_reviews = !ls -1 -d aclImdb/train/neg/*
unsupervised_reviews = !ls -1 -d aclImdb/train/unsup/*

In [ ]:
print('Positive #',len(positive_reviews))
print('Negative #',len(negative_reviews))
print('Unsupervised #',len(unsupervised_reviews))

Positive # 12500
Negative # 12500
Unsupervised # 50000


In [ ]:
positive_reviews[0]

'aclImdb/train/pos/0_9.txt'

So, we have 12500 reviews for positive and negative classifications, and 50000 that are unlabeled.  

In [ ]:
vocabulary = set()

with open('./aclImdb/imdb.vocab') as vocab_file:
  for word in vocab_file:
    vocabulary.add(word.rstrip())


In [ ]:
with open(positive_reviews[0]) as review:
  for line in review:
    for word in line.rstrip().split(' '):
      if word not in vocabulary:
        print(word)
        

Bromwell
High
comedy.
It
life,
"Teachers".
My
35
Bromwell
High's
"Teachers".
The
financially,
teachers'
pomp,
situation,
I
students.
When
I
school,
I
.........
..........
High.
A
line:
INSPECTOR:
I'm
teachers.
STUDENT:
Welcome
Bromwell
High.
I
Bromwell
High
fetched.
What
isn't!


Looking at the words in a review that don't show up in the vocab, we need to handle punctuation in a more delicate way (also, we will be making everything lower cased).  For this, we will turn to a standard modern NLP library spaCy.  spaCy is a library that handles a number of different low-level NLP tasks like tokenization, part-of-speech recognition, and named entity recognition.  For now, we will be focusing on the tokenization aspect.



In [ ]:
from spacy.lang.en import English
from tqdm import tqdm_notebook
nlp = English()
# Create a Tokenizer with the default settings for English
# including punctuation rules and exceptions
tokenizer = nlp.Defaults.create_tokenizer(nlp)
counts = {}
for review in positive_reviews + negative_reviews:
  with open(review) as review:
    for line in review:
      for word in tokenizer(line.rstrip().lower()):
        if word.text not in vocabulary:
          pass
        else:
          counts[word.text] = counts.get(word.text,0) + 1




In [ ]:
count2words = {}
for word in counts:
  count = counts[word]
  if count not in count2words:
    count2words[count] = []
  count2words[count].append(word)


In [ ]:
running_total = 0
vocabulary = []
for count in reversed(sorted(count2words)):
  running_total += len(count2words[count])
  vocabulary += count2words[count]
  if running_total > 10000:
    break

In [ ]:
print(vocabulary)

['the', 'and', 'a', 'of', 'to', 'is', 'it', 'in', 'i', 'this', 'that', 'was', 'as', 'for', 'with', 'movie', 'but', 'film', 'on', 'you', "n't", 'not', 'are', 'he', 'his', 'have', 'be', 'one', 'all', 'at', 'they', 'by', '!', 'an', 'who', 'from', 'like', 'so', 'her', 'or', 'just', 'there', 'do', 'about', 'has', 'out', 'if', 'what', 'some', 'good', '?', 'she', 'more', 'very', 'when', 'would', 'up', 'no', 'even', 'time', 'can', 'my', 'which', 'only', 'story', 'really', 'had', 'their', 'see', 'were', 'we', 'me', 'did', 'does', 'well', 'than', 'much', 'could', 'get', 'been', 'will', 'people', 'bad', 'into', 'other', 'because', 'great', 'also', 'first', 'how', 'most', 'him', 'made', 'its', 'make', 'then', 'way', 'them', 'too', 'any', 'movies', 'after', 'br', 'think', 'characters', 'character', 'two', 'watch', 'films', 'many', 'seen', 'being', 'never', 'plot', 'life', 'little', 'love', 'acting', 'where', 'best', 'over', 'show', 'know', 'off', 'ever', 'man', 'your', 'better', 'still', 'here', 'e

We can now tokenize our text and the things we ignore are *mostly* puncutation (most tokenizers will split contractions like I'm into I and 'm which happens here, but the original vocab doesn't expect -- we will just let that happen here)


#Step 1

Fill out the function below to:

* Tokenize a document
* Extract all n-grams and their counts of the given order

* It's ok to utilize your work from the first assignment here (although use SpaCy and go to lower case)

In [ ]:
tokenizer = nlp.Defaults.create_tokenizer(nlp)
import pandas as pd

def get_n_grams(filename,n):
  ngrams = {}


  if n == 1:
    with open(filename) as review:
      for line in review:
        for word in tokenizer(line.rstrip().lower()):
          if word.text not in vocabulary:
            pass
          else:
            ngrams[word.text] = ngrams.get(word.text,0) + 1



  else:
    with open(filename) as review:
      for line in review:
        line = tokenizer(line.rstrip().lower()).text.split()
        for pair in zip(*(line[i:] for i in range(n))):
          if n == 2:
            if pair in vocab_bigrams:
              ngrams[pair] = ngrams.get(pair,0)+1
          elif n == 3:
            if pair in vocab_trigrams:
              ngrams[pair] = ngrams.get(pair,0)+1



  return ngrams



With the ability to extract our n-grams, we will now be constructing a linear classifier.  To do this, we will need to set up our data.  

#Step 2

* Make a $|D|\times|unigrams|$ matrix, `X_unigrams` that contains all of the unigram counts for the documents -- each row should be a document and each column should be a unigram
* Make a $|D|\times1$ matrix, `Y` that contains the ratings for the documents -- we will say that a positive review has a rating of `1` and a negative review is `0`

In [ ]:
from numpy import *
import numpy as np
X = []
Y = []
vocab_dict = {k: v for v, k in enumerate(vocabulary)}

for review in tqdm_notebook(positive_reviews+negative_reviews):
  rowX = [0] * len(vocabulary)
  rowY = []

  with open(review) as review:
    if 'pos' in review.name:
      rowY.append(1)
    else:
      rowY.append(0)


    name = review.name
    unigrams = get_n_grams(name,1)
    for k,v in unigrams.items(): 
      x_pos = vocab_dict[k]
      rowX[x_pos] = v


  X.append(rowX)
  Y.append(rowY)

X_np = np.array(X, dtype=int)
Y_np = np.array(Y,dtype=int)


/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:7: TqdmDeprecationWarning: This function will be removed in tqdm==5.0.0
Please use `tqdm.notebook.tqdm` instead of `tqdm.tqdm_notebook`
  import sys


We are now going to use a neural network to do our regression, and we are going to use Torch.


In [ ]:
import torch
import torch.nn
from torch.autograd import Variable

We will first construct a module that does what we want.  In torch you implement modules and you define the forward pass of the model (Torch uses autograd to automatically compute the backward pass based on the forward pass). 

#Step 3
* Implement the linear regression using Torch
* You will want to use a Linear layer (it is essentially the same as the weights we learned above)  -- think about what the input dimension and output dimensions should be (how big is our input, how many things are we predicting)


In [ ]:
import torch.nn as nn
class LinearRegression(nn.Module):
  def __init__(self, size):
    super(LinearRegression, self).__init__()
    self.linear = nn.Linear(size,1)
  def forward(self,X):
    out = self.linear(X)
    return out

With the above model, we now need to actually train it.  We are going to use a pretty simple training regimen here.

In [ ]:
import torch.optim as optim
from tqdm import tqdm_notebook

def train(X, Y, model, batch_size, epochs, learning_rate, loss_criterion):
  optimizer = optim.Adam(model.parameters(), lr=learning_rate)
  for e in tqdm_notebook(range(epochs)):
    for batch in range(0,X.size(0),batch_size):
      optimizer.zero_grad()         # zero the gradient buffers
      output = model(X[batch:(batch)+batch_size,:]) 
      loss = criterion(output, Y[batch:(batch)+batch_size,:])
      loss.backward()               # run the loss in the backward direction
      nn.utils.clip_grad_norm_(model.parameters(), 3.0) # clip the gradients for stability
      optimizer.step()              # update the weights

    if e % 100 == 0:
      print('epoch {}, loss {}'.format(e,loss.item()))

#Step 4 
* Construct your model 
* In torch, if you want to use a GPU we have to tell torch to move the model (and parameters to the gpu) so you need to call `.to('cuda')` (Note this does not change the original model, it returns a model on the gpu so you probably want to do something like `model = model.to('cuda')`
*  As a note, you also have to do this for the data -- to get the data from numpy you will want to do something like `X_tensor = torch.Tensor(X).to('cuda')`  
* Call the above training code -- watch how the loss changes as the model trains -- experiment with different training hyperparameters -- learning rate and epoch -- see when the model hits a minima
* For a linear regression, we are using Mean Square Error as our loss `torch.nn.MSELoss`

In [ ]:
#Training linear regression model
model = LinearRegression(len(vocabulary))
X = Variable(torch.from_numpy(X_np).type(torch.FloatTensor))
Y = Variable(torch.from_numpy(Y_np).type(torch.FloatTensor))

if torch.cuda.is_available():
  X = X.cuda()
  Y = Y.cuda()
  model = model.cuda()

epochs = 400
batch_size = 12500
criterion = torch.nn.MSELoss()
learning_rate = 0.01

prediction = train(X,Y,model,batch_size,epochs, learning_rate, criterion)


/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:6: TqdmDeprecationWarning: This function will be removed in tqdm==5.0.0
Please use `tqdm.notebook.tqdm` instead of `tqdm.tqdm_notebook`
  


epoch 0, loss 7.624569892883301
epoch 100, loss 0.22561417520046234
epoch 200, loss 0.21419194340705872
epoch 300, loss 0.28099095821380615



###Question 1
After the model has finished training, compare the results to the original linear regression -- how do they compare?

With that, let's make a nonlinear regression

#Step 5
* Implement the class below -- you should have two linear weights  (Input -> Hidden), (Hidden -> Output)
* In your forward, make sure you apply a nonlinear activation function after going from (Input -> Hidden) before (Hidden -> Output) -- you can choose the nonlinear activation (sigmoid, tanh, relu, leaky-relu are all fine)
* Train the model

###Question 2
How does this model compare to the above models?

In [ ]:
class NonLinearRegression(nn.Module):
  def __init__(self, size,hidden_layers):
    super(NonLinearRegression, self).__init__()

    self.hidden = torch.nn.Linear(size,hidden_layers)
    self.predict = torch.nn.Linear(hidden_layers,1)
  def forward(self,X):
    X = torch.sigmoid(self.hidden(X))
    X = self.predict(X)
    return X

Ok, we have a linear regression and a non-linear regression -- let's do a Logistic Regression and non-linear Logistic regression now:
#Step 6
* Implement a standard logistic regression and a logistic regression with a hidden layer and non-linear activation
* These should be very similar to above except they should have `torch.Sigmoid` applied to the output
* Note, `MSELoss` is no longer applicable here -- we need to use Binary Cross Entropy Loss `torch.nn.BCELoss`

In [ ]:
class LogisticRegression(nn.Module):
  def __init__(self, size):
    super(LogisticRegression, self).__init__()
    self.weight = nn.Linear(size,1)
  def forward(self,X):
    y = torch.sigmoid(self.weight(X))
    return y


class LogisticANN(nn.Module):
  def __init__(self, size):
    super(LogisticANN, self).__init__()
    hidden_layers = 10
    self.weight = torch.nn.Linear(size,hidden_layers)
    self.predict = torch.nn.Linear(hidden_layers,1)
  def forward(self,X):
    y = torch.sigmoid(self.weight(X))
    y = self.predict(y)
    return y

# Step 7
* So far we have just been limited to unigrams
* In this part, let's open it up
* You are going to do some model exploration -- try out 
 * different configurations of inputs (different order n-grams, combinations of n-grams)
 * different numbers and sizes of input layers
 * different non-linear activations

* Normally, you should pick the model that does the best not on training data, but on the evaluation data.  Since this dataset doesn't come with an evaluation set, we will instead be using the test set as our evaluation set (this is obviously poor practice, and should not be done in the real world).  Or rather we will be using a portion of the test files as evaluation.

* Pick a model architecture, train it, run it on the eval data 
* Do this multiple times, keep the best model 

#### Question 3:
How do you pick the "best" model -- what metric do you want to use here?

* As a note, there will be n-grams that show up in the evaluation and training sets that don't show up in the training data, you will have to be able to account for this (of course, ignoring is acceptable) but your code shouldn't fail


## Bonus - Grid Search (1 pt)
* Instead of manually testing models, run a grid search over the hyperparameter space (model architecture, input, other training hyper parameters) -- keeping the best model

* When you are confident in your model, run it on the held out test set -- show the results below


In [ ]:
positive_raw_test = !ls -1 -d aclImdb/train/pos/*
negative_raw_test = !ls -1 -d aclImdb/train/neg/*

In [ ]:
positive_eval = positive_raw_test[:len(positive_raw_test)//4]
negative_eval = negative_raw_test[:len(negative_raw_test)//4]
positive_test = positive_raw_test[len(positive_raw_test)//4:]
negative_test = negative_raw_test[len(negative_raw_test)//4:]

In [ ]:
#Get vocabulary for bigrams and trigrams
vocab_bigrams = {}
vocab_trigrams = {}
for review in tqdm_notebook(positive_reviews+negative_reviews):
  with open(review) as review:
    with open(review.name) as review:
      for line in review:
        line = tokenizer(line.rstrip().lower()).text.split()
        for pair in zip(*(line[i:] for i in range(2))):
          vocab_bigrams[pair] = vocab_bigrams.get(pair,0)+1

        for pair in zip(*(line[i:] for i in range(3))):
          vocab_trigrams[pair] = vocab_trigrams.get(pair,0)+1

vocab_bigrams = {key:val for key, val in vocab_bigrams.items() if val >= 50}
vocab_trigrams = {key:val for key, val in vocab_trigrams.items() if val >= 30}
print(len(vocab_bigrams))
print(len(vocab_trigrams))


/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:5: TqdmDeprecationWarning: This function will be removed in tqdm==5.0.0
Please use `tqdm.notebook.tqdm` instead of `tqdm.tqdm_notebook`
  """



10637
7998


In [ ]:
import torch.optim as optim


def trainForEval(X,Y,model,batch_size,epochs,criterion):
  optimizer = optim.SGD(model.parameters(), lr=0.001)
  for e in range(epochs):
    for batch in range(0,X.size(0),batch_size):
      optimizer.zero_grad()         # zero the gradient buffers
      output = model(X[batch:(batch)+batch_size,:]) 
      loss = criterion(output, Y[batch:(batch)+batch_size,:])
      loss.backward()               # run the loss in the backward direction
      nn.utils.clip_grad_norm_(model.parameters(), 3.0) # clip the gradients for stability
      optimizer.step()              # update the weights


In [ ]:
from numpy import *
import numpy as np
from tqdm.notebook import tqdm
from tqdm import tqdm_notebook

def storeData(pos,neg,n,vocabulary):
  X = []
  Y = []
  vocab_dict = {k: v for v, k in enumerate(vocabulary)}
  for review in tqdm_notebook(pos+neg):
    rowX = [0] * len(vocabulary)
    rowY = []

    with open(review) as review:
      if 'pos' in review.name:
        rowY.append(1)
      else:
        rowY.append(0)
      ngrams = get_n_grams(review.name,n)
      for k,v in ngrams.items(): 
        x_pos = vocab_dict[k]
        rowX[x_pos] = v


    X.append(rowX)
    Y.append(rowY)

    
  X_np = np.array(X, dtype=int)
  Y_np = np.array(Y,dtype=int)
  X = Variable(torch.from_numpy(X_np).type(torch.FloatTensor))
  Y = Variable(torch.from_numpy(Y_np).type(torch.FloatTensor))
  return X,Y


In [ ]:

x_train_uni, y_train_uni = storeData(positive_test,negative_test,1,vocabulary)
x_pred_uni, y_pred_uni= storeData(positive_eval,negative_eval,1,vocabulary)
x_train_bi, y_train_bi = storeData(positive_test,negative_test,2,vocab_bigrams)
x_pred_bi, y_pred_bi = storeData(positive_eval,negative_eval,2,vocab_bigrams)
x_train_tri, y_train_tri = storeData(positive_test,negative_test,3,vocab_trigrams)
x_pred_tri, y_pred_tri = storeData(positive_eval,negative_eval,3,vocab_trigrams)

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:10: TqdmDeprecationWarning: This function will be removed in tqdm==5.0.0
Please use `tqdm.notebook.tqdm` instead of `tqdm.tqdm_notebook`
  # Remove the CWD from sys.path while we load stuff.


In [ ]:
#Model selection

import sklearn.metrics as sm
#N-grams: Unigrams, bigrams, trigrams

for i in range(3):
  h = 10
  if i == 0:
    v = vocabulary
    x_train = x_train_uni
    y_train = y_train_uni
    x_pred = x_pred_uni
    y_pred = y_pred_uni
  elif i == 1:
    v = vocab_bigrams
    x_train = x_train_bi
    y_train = y_train_bi
    x_pred = x_pred_bi
    y_pred = y_pred_bi
  elif i == 2:
    v = vocab_trigrams
    x_train = x_train_tri
    y_train = y_train_tri
    x_pred = x_pred_tri
    y_pred = y_pred_tri

  #Linear regression model
  model = LinearRegression(len(v))
  criterion = torch.nn.MSELoss()
  trainForEval(x_train,y_train,model,b,e,criterion)
  y_test = model(x_pred)
  print("Linear Regression, {}-grams, {} batches, {} epochs: ".format(i+1,b,e))
  print("Mean squared error =", round(sm.mean_squared_error(y_test.detach().numpy(), y_pred.detach().numpy()), 2)) 
  print()

  #Non linear regression model
  model = NonLinearRegression(len(v),h)
  criterion = torch.nn.MSELoss()
  trainForEval(x_train,y_train,model,b,e,criterion)
  y_test = model(x_pred)
  mean_sq = sm.mean_absolute_error(y_test.detach().numpy(),y_pred.detach().numpy())
  print("Non linear regression, {}-grams, {} hidden layers, {} batches, {} epochs: ".format(i+1,h,b,e))
  print("Mean squared error =", round(sm.mean_squared_error(y_test.detach().numpy(), y_pred.detach().numpy()), 2)) 
  print()

  #Logistic Regression model
  model = LogisticRegression(len(v))
  criterion = torch.nn.BCELoss()
  trainForEval(x_train,y_train,model,b,e,criterion)
  y_test = model(x_pred)
  mean_sq = sm.mean_absolute_error(y_test.detach().numpy(),y_pred_uni.detach().numpy())
  print("Logistic Regression, {}-ngrams, {} batches, {} epochs: ".format(i+1,b,e))
  print("Mean squared error =", round(sm.mean_squared_error(y_test.detach().numpy(), y_pred.detach().numpy()), 2)) 
  print()

  #Logistic ANN Regression model
  model = LogisticANN(len(v))
  criterion = torch.nn.BCEWithLogitsLoss()
  trainForEval(x_train,y_train,model,b,e,criterion)
  y_test = model(x_pred)
  mean_sq = sm.mean_absolute_error(y_test.detach().numpy(),y_pred_uni.detach().numpy())
  print("Logistic ANN, {}-grams, {} batches, {} epochs: ".format(i+1,b,e))
  print("Mean squared error =", round(sm.mean_squared_error(y_test.detach().numpy(), y_pred.detach().numpy()), 2)) 
  print()



# By testing all combinations of unigram, bigram and trigram models for the 4 models we created, is determined that 
# The logistic model using unigrams is the best model with a MSE of 0.25

Linear Regression, 1-grams, 25000 batches, 10 epochs: 
Mean squared error = 0.34

Non linear regression, 1-grams, 10 hidden layers, 25000 batches, 10 epochs: 
Mean squared error = 0.35

Logistic Regression, 1-ngrams, 25000 batches, 10 epochs: 
Mean squared error = 0.25

Logistic ANN, 1-grams, 25000 batches, 10 epochs: 
Mean squared error = 0.42

Linear Regression, 2-grams, 25000 batches, 10 epochs: 
Mean squared error = 0.42

Non linear regression, 2-grams, 10 hidden layers, 25000 batches, 10 epochs: 
Mean squared error = 0.25

Logistic Regression, 2-ngrams, 25000 batches, 10 epochs: 
Mean squared error = 0.25

Logistic ANN, 2-grams, 25000 batches, 10 epochs: 
Mean squared error = 0.88

Linear Regression, 3-grams, 25000 batches, 10 epochs: 
Mean squared error = 0.49

Non linear regression, 3-grams, 10 hidden layers, 25000 batches, 10 epochs: 
Mean squared error = 0.39

Logistic Regression, 3-ngrams, 25000 batches, 10 epochs: 
Mean squared error = 0.25

Logistic ANN, 3-grams, 25000 batc

## Bonus: Augment with unsupervised data (up to 3 points)
* The dataset comes with more unsupervised data than supervised -- come up with a way to augment your training set with the unsupervised data, such that your model performance improves (to do this comparison keep model architecture the same, and compare the augmented data model vs the original on the evaluation set)